In [ ]:
import cv2
import numpy as np
from playsound import playsound

cap = cv2.VideoCapture(1)

while cap.isOpened():
    ret, frame = cap.read()

    #to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    #adaptive thresholding
    adaptive_threshold = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2
    )

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(
        adaptive_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
# Filter contours based on area
    min_contour_area = 1000  # Adjust as needed
    contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]

    total_area = frame.shape[0] * frame.shape[1]

    change_area = sum(cv2.contourArea(cnt) for cnt in contours)

    percentage = (change_area / total_area) * 100
    print(percentage)

    threshold_percentage = 98  # Adjust this value as needed

    if percentage > threshold_percentage:
        print(f"Detected changes occupy more than {threshold_percentage}% of the screen. Possible wall detected.")
        playsound('w.mp3')
        
    text = f"Percentage: {percentage:.2f}%"
    cv2.putText(
        adaptive_threshold,
        text,
        (100, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (255,0,0),
        1,
        cv2.LINE_AA,
    )
    cv2.imshow('Adaptive Threshold', adaptive_threshold)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()